# Services (daily)

In [256]:
import pandas as pd

## Database connection

In [257]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [258]:
# Load dimensions
time_dimension = pd.read_sql_table("TIME_DIMENSION", OLAP_connection)
courier_dimension = pd.read_sql_table("COURIER_DIMENSION", OLAP_connection)
customer_dimension = pd.read_sql_table("CUSTOMER_DIMENSION", OLAP_connection)
office_dimension = pd.read_sql_table("OFFICE_DIMENSION", OLAP_connection)
service_status_dimension = pd.read_sql_table("SERVICE_STATUS_DIMENSION", OLAP_connection)

In [259]:
# Load fact table data from OLTP
services = pd.read_sql_table("mensajeria_servicio", OLTP_connection)
service_statuses = pd.read_sql_table("mensajeria_estadosservicio", OLTP_connection)
users_information = pd.read_sql_table("clientes_usuarioaquitoy", OLTP_connection)

## Transformation

In [260]:
services = services.dropna(subset=["mensajero_id"])

In [261]:
services = services[[
		"id",
		"fecha_solicitud",
		"cliente_id",
		"mensajero_id",
		"usuario_id"
	]]

In [262]:
latest_statuses = service_statuses.sort_values("fecha").drop_duplicates(
		"servicio_id", keep="last"
)

In [263]:
latest_statuses = latest_statuses[["estado_id", "servicio_id"]]

In [264]:
latest_statuses = latest_statuses.rename(
		columns={
				"id": "status_record_id",
				"estado_id": "status_id",
				"servicio_id": "service_id"
		}
)

In [265]:
services = services.rename(
		columns={
				"id": "service_id",
				"fecha_solicitud": "date",
				"cliente_id": "original_customer_id",
				"mensajero_id": "original_courier_id"
		}
)

In [266]:
services = services.merge(
		latest_statuses, on="service_id", how="left"
	)

In [267]:
services = services[services["status_id"] >= 5] # Delivered

In [268]:
services = services.merge(users_information[["id", "sede_id"]], left_on="usuario_id", right_on="id", how="inner")
services.drop(columns=["usuario_id", "id"], inplace=True)

In [269]:
services = services.merge(office_dimension[["office_id", "original_office_id"]], left_on="sede_id", right_on="original_office_id", how="inner")
services.drop(columns=["sede_id", "original_office_id"], inplace=True)

In [270]:
services = pd.merge(services, time_dimension[["time_id", "date"]], on="date", how="inner")
services.drop(columns=["date", "status_id"], inplace=True)

In [271]:
services = services.groupby(["original_customer_id", "original_courier_id", "time_id", "office_id"]).size().reset_index(name="total_services")

In [272]:
services.head()

,original_customer_id,original_courier_id,time_id,office_id,total_services
0,2,2.0,233280,15,1
1,2,7.0,200160,15,1
2,2,7.0,201600,15,1
3,2,7.0,204480,15,1
4,2,7.0,214560,15,1


In [273]:
services[["total_services"]].sum()

total_services    10108
dtype: int64

In [274]:
courier_dimension

,courier_id,original_courier_id,courier_city
0,0,1,ACOPI YUMBO
1,1,42,CALI
2,2,48,CALI
3,3,41,CALI
4,4,13,PASTO
5,5,28,CALI
6,6,33,TULUA
7,7,36,CALI
8,8,8,PALMIRA
9,9,21,CALI


In [275]:
service_fact = services.merge(customer_dimension[["customer_id", "original_customer_id"]], on="original_customer_id", how="inner")
service_fact.drop(columns=["original_courier_id"], inplace=True)

In [277]:
service_fact = services.merge(courier_dimension[["courier_id", "original_courier_id"]], on="original_courier_id", how="inner")
service_fact.drop(columns=["original_courier_id"], inplace=True)

In [223]:
service_fact.reset_index(inplace=True)
service_fact = service_fact.rename(columns={ "index": "service_daily_id" })
service_fact.set_index("service_daily_id", inplace=True)

## Load

In [279]:
from sqlalchemy import BigInteger


dtype_mapping = {
	"time_id": BigInteger(),
	"customer_id": BigInteger(),
	"courier_id": BigInteger(),
	"office_id": BigInteger(),
}

In [280]:
# Save the fact table to OLAP
service_fact.to_sql(
	"SERVICE_DAILY_FACT_TABLE", OLAP_connection, if_exists="replace", index=True, dtype=dtype_mapping
)

703